In [1]:
import torch
import numpy as np
import sys

sys.path.append('human_body_prior/tools/')
from omni_tools import copy2cpu as c2c
from os import path as osp

both57m_dir = 'data/both57m/'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(comp_device)

cpu


In [2]:
sys.path.append('human_body_prior/body_model/')
from body_model import BodyModel

gender = "male"

bm_fname = 'smplh_models/{}/model.npz'.format(gender)
bm = BodyModel(bm_fname=bm_fname).to(comp_device)
faces = c2c(bm.f)

In [3]:
data_number = 500

both57m_npz = osp.join(both57m_dir, str(data_number), 'data.npz')
bdata = np.load(both57m_npz)
len = bdata['seq_len']

print('Data keys available:%s\n'%list(bdata.keys()))
print(bdata['full_annot'], '\n')

motion_parms = {
    'root_orient': torch.Tensor(bdata['root_orient']).to(comp_device),
    'pose_body': torch.Tensor(bdata['body_pose'].reshape(len,-1)).to(comp_device),
    'pose_hand': torch.Tensor(bdata['hand_pose'].reshape(len,-1)).to(comp_device),
    'trans': torch.Tensor(bdata['trans']).to(comp_device)
}

print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in motion_parms.items()])))

Data keys available:['trans', 'body_pose', 'root_orient', 'hand_pose', 'seq_len', 'framerate', 'full_annot', 'hand_annot']

['a person crosses their hands in front of their body to indicate not to do it.'
 'the person shakes their head and waves their hand to indicate that this is not right.'
 'when a person expresses refusal, they cross their arms in front of their chest.'] 

Body parameter vector shapes: 
root_orient: torch.Size([150, 3]) 
pose_body: torch.Size([150, 63]) 
pose_hand: torch.Size([150, 90]) 
trans: torch.Size([150, 3])


In [4]:
import trimesh
sys.path.append('body_visualizer')
from tools.vis_tools import colors
from mesh.mesh_viewer import MeshViewer
from mesh.sphere import points_to_spheres
from tools.vis_tools import show_image

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

AttributeError: dlsym(0x3c02ed3b0, eglGetCurrentContext): symbol not found

In [ ]:
body_pose = bm(**{k:v for k,v in motion_parms.items() if k in ['pose_body', 'pose_hand', 'root_orient']})

def vis_body_hand_pose(fId = 0, SHOW=True):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    if SHOW: show_image(body_image)

    return body_image

body_images = []
for i in range(len):
    body_images.append(vis_body_hand_pose(fId=i, SHOW=False))

In [ ]:
import mediapy as media

media.show_video(images=body_images,fps=int(bdata['framerate']),width=700)